In [14]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import iirnotch, lfilter, filtfilt, sawtooth, square

def odstrani_kvadratni_signal(s: np.ndarray, f: float, fvz: int) -> np.ndarray:
    nyquist = 0.5 * fvz
    harmonics = np.arange(1, fvz // (2 * f), 2) * f
    filtered_signal = s.copy()

    for h_freq in harmonics:
        bandwidth = h_freq * 0.1
        Q = h_freq / bandwidth
        b, a = iirnotch(h_freq, Q, fvz)
        filtered_signal = filtfilt(b, a, filtered_signal)

    return filtered_signal

if __name__ == '__main__':
    fvz = 1000
    t = np.arange(0, 2, 1 / fvz)
    sig = np.sin(2 * np.pi * t)
    #duty=(sig + 1)/2
    s_kvadratni = square(2 * np.pi * 30 * t)
    s_zagasti = sawtooth(2 * np.pi * 15 * t)
    s_vhodni = s_kvadratni + s_zagasti
    s_filtrirani = odstrani_kvadratni_signal(s_vhodni, 30, fvz)

    plt.figure()
    plt.subplot(4, 1, 1)
    plt.plot(t, s_kvadratni)
    plt.title('Kvadratni signal')
    plt.xlabel('Čas [s]')
    plt.ylabel('Amplituda')

    plt.subplot(4, 1, 2)
    plt.plot(t, s_zagasti)
    plt.title('Kvadratni signal')
    plt.xlabel('Čas [s]')
    plt.ylabel('Amplituda')

    plt.subplot(4, 1, 3)
    plt.plot(t, s_vhodni)
    plt.title('Vhodni signal')
    plt.xlabel('Čas [s]')
    plt.ylabel('Amplituda')

    plt.subplot(4, 1, 4)
    plt.plot(t, s_zagasti, label='Žagasti signal')
    plt.plot(t, s_filtrirani, label='Filtrirani signal')
    plt.legend()
    plt.title('Žagasti in filtrirani signal')
    plt.xlabel('Čas [s]')
    plt.ylabel('Amplituda')

    plt.tight_layout()
    plt.show()

In [26]:
import numpy as np
from pydub import AudioSegment
from scipy.fftpack import fft
from scipy.io.wavfile import read, write
from scipy.signal import iirnotch, filtfilt, butter, sosfiltfilt

def load_mp3(file_path):
    audio = AudioSegment.from_mp3(file_path)
    audio = audio.set_channels(1)
    samples = np.array(audio.get_array_of_samples())
    return audio.frame_rate, samples

def find_frequencies(signal, rate, threshold=0.05):
    N = len(signal)
    T = 1.0 / rate

    yf = fft(signal)
    xf = np.linspace(0.0, 1.0 / (2.0 * T), N // 2)

    peaks = []
    max_val = 2.0 / N * np.abs(yf[:N // 2]).max()
    for i in range(1, N // 2 - 1):
        if (
            2.0 / N * np.abs(yf[i]) > threshold * max_val
            and yf[i - 1] < yf[i]
            and yf[i] > yf[i + 1]
        ):
            peaks.append(xf[i])

    return peaks

def butter_bandstop_filter(f_low, f_high, fs, order=5):
    nyquist = 0.5 * fs
    low = f_low / nyquist
    high = f_high / nyquist
    sos = butter(order, [low, high], btype='bandstop', analog=False, output='sos')
    return sos

def odstrani_signal(s: np.ndarray, ringing_freqs: np.ndarray, fvz: int) -> np.ndarray:
    filtered_signal = s.copy()

    for idx, freq in enumerate(ringing_freqs):
        print(f"Obdelava frekvence {idx + 1} od {len(ringing_freqs)}: {freq} Hz")
        bandwidth = freq * 0.5
        f_low = freq - bandwidth / 2
        f_high = freq + bandwidth / 2
        sos = butter_bandstop_filter(f_low, f_high, fvz)
        filtered_signal = sosfiltfilt(sos, filtered_signal)

    return filtered_signal


if __name__ == '__main__':

    rate_zvonenje, data_zvonenje = load_mp3("Old-phone-ringtone.mp3")

    ringing_freqs = find_frequencies(data_zvonenje, rate_zvonenje)

    print("Najdene frekvence zvonjenja:", ringing_freqs)

    rate_himna, data_himna = read("portugalska_himna_zvonenje.wav")

    filtered_data = odstrani_signal(data_himna, ringing_freqs, rate_himna)

    # Shranite filtrirani avdio posnetek
    write("zvonenje.wav", rate_himna, filtered_data.astype(np.int16))


Najdene frekvence zvonjenja: [1318.8707205276655, 1319.5114155582837, 1321.1131531348294, 1322.0741956807567, 1322.3945431960658, 1322.8750644690294, 1323.1954119843385, 1324.3166282879206, 1324.7971495608842, 1325.2776708338479, 1326.2387133797754, 1327.3599296833572, 1673.3352462172118, 1675.5776788243757, 1677.3395901585757, 1677.980285189194, 1678.4608064621577, 1679.5820227657396, 1680.0625440387032, 1680.7032390693216, 1681.5041078575944, 1681.984629130558, 1682.6253241611762, 1683.2660191917946, 1683.7465404647583, 1684.547409253031, 1685.0279305259946, 1685.668625556613, 1686.1491468295767, 1686.7898418601949, 1687.430536890813, 1688.231405679086, 1688.7119269520497, 1689.3526219826679, 1689.672969497977, 1690.47383828625, 1691.4348808321772, 1695.1188772582323, 1695.7595722888504, 1696.8807885924323, 1698.0020048960143, 1699.6037424725598, 1700.2444375031782, 3478.173147468819, 4386.678700885496, 4387.15922215846, 4388.120264704387, 4389.722002280932, 4390.362697311551, 4391.0